In [1]:
import GenFunction
import DataGenerator
import FFmpegDecoder

In [ ]:
### generate N images per class ###

In [ ]:
DataGenerator.generate(GenFunction.mix2image, 200, "Base_t", "train")
DataGenerator.generate(GenFunction.mix2image, 50, "Base_v", "valid")

In [ ]:
DataGenerator.generate(GenFunction.mix4image, 200, "Base_t", "train")
DataGenerator.generate(GenFunction.mix4image, 50, "Base_v", "valid")

In [ ]:
DataGenerator.generate(GenFunction.concat2image, 200, "Base_t", "train")
DataGenerator.generate(GenFunction.concat2image, 50, "Base_v", "valid")

In [ ]:
### decoding video ###

In [ ]:
# X.mp4 to DIR/XXXX.jpg
FFmpegDecoder.video_to_image("sample.mp4", "sample_images", 30)

In [ ]:
# DIR1/XXXX.jpg and DIR1/YYYY.jpg to DIR2/XXXX.jpg
DataGenerator.frame_to_input(GenFunction.concat2image, "sample_images", "input")

In [ ]:
# default setting
from tensorflow import keras

In [2]:
# AMD setting
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ["PLAIDML_EXPERIMENTAL"] = "1"
os.environ["PLAIDML_DEVICE_IDS"] = "opencl_amd_ellesmere.0"
import keras

Using plaidml.keras.backend backend.


In [3]:
import KerasFunction as K
import VGGModel

In [4]:
K.keras = keras
VGGModel.keras = keras
K.gen_f = GenFunction.concat2image

In [5]:
model = K.make_model()
model.summary()

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_2_ (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 7, 7, 512)    14714688    input_1[0][0]                    
__________________________________________________________________________________________________
vgg16_ (Model)                  (None, 7, 7, 512)    14714688    input_2_[0][0]                   
__________________________________________________________________________________________________
multiply_1

In [6]:
# Make new model with random weights
model = K.make_model()
K.compile_model(model)

# Train model
K.train_model(model, train_steps = 50, valid_steps=10, epochs=2)

# Save model
K.save_model("model.h5", model)

Epoch 1/2
Found 200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.
50/50 [==============================] - 66s 1s/step - loss: 3.2886 - acc: 0.5038 - val_loss: 0.7099 - val_acc: 0.6937
Epoch 2/2
50/50 [==============================] - 58s 1s/step - loss: 0.5306 - acc: 0.8063 - val_loss: 0.7031 - val_acc: 0.7368


In [10]:
# Make new model with saved weights
model = K.load_model("model.h5")
K.compile_model(model)

# Check validation
(_, data_gen) = K.get_data_gen(200)
K.check_validation(model, data_gen)

Found 200 images belonging to 4 classes.
              precision    recall  f1-score   support

         0.0       1.00      0.62      0.77        50
         1.0       0.97      0.58      0.72        50
         2.0       0.57      0.86      0.68        50
         3.0       0.65      0.82      0.73        50

    accuracy                           0.72       200
   macro avg       0.80      0.72      0.72       200
weighted avg       0.80      0.72      0.72       200

[[31  0 14  5]
 [ 0 29 10 11]
 [ 0  1 43  6]
 [ 0  0  9 41]]


In [ ]:
# Make new model with saved weights
model = K.load_model("model.h5")
K.compile_model(model)

In [ ]:
# detect from images
K.detection(model, "input")

In [ ]:
#model = K.load_model("model.h5")
#K.compile_model(model)
K.detection(model, "valid/3_Fade")